
# 🧭 Colab 3 — DPO with **LoRA** on **SmolLM2‑135M** (FP32 • Stable)



Assignment Done By :- ***Dev Mulchandani***

## 0) Install (run both cells, then **Runtime → Restart runtime**)

In [1]:

!pip uninstall -y trl transformers accelerate datasets peft bitsandbytes wandb diffusers || true


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: wandb 0.22.3
Uninstalling wandb-0.22.3:
  Successfully uninstalled wandb-0.22.3
Found existing installation: diffusers 0.35.2
Uninstalling diffusers-0.35.2:
  Successfully uninstalled diffusers-0.35.2


In [2]:

!pip install --no-cache-dir "trl==0.7.1" "transformers==4.35.2" "accelerate==0.24.1" "datasets==2.16.1" "peft==0.7.1" "numpy==1.26.4"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 195.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 388.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 217.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 404.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 379.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 251.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 379.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 434.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

## 1) Check GPU

In [1]:

!nvidia-smi || echo "No GPU detected — In Colab: Runtime > Change runtime type > GPU"


Mon Nov 10 01:44:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2) Disable W&B and import libraries

In [2]:

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_SILENT"]  = "true"
os.environ["WANDB_MODE"]    = "offline"
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"  # ensure no AMP

import transformers, torch, sys, numpy as np, gc
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import DPOTrainer

print("Python:", sys.version.split()[0])
print("Transformers:", transformers.__version__)
print("TRL:", __import__("trl").__version__)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Python: 3.12.12
Transformers: 4.35.2
TRL: 0.7.1
Torch: 2.8.0+cu126
CUDA available: True


## 3) Tiny preference dataset (prompt, chosen, rejected)

In [3]:

raw = [
    {"prompt":"Explain what a function is in Python.",
     "chosen":"A function is a reusable block of code defined with `def` that can take parameters and often returns a value with `return`.",
     "rejected":"A function is when the computer thinks really hard and things happen by themselves."},
    {"prompt":"Give two tips to study better.",
      "chosen":"Use active recall in short sessions, and space practice across days. Sleep well to consolidate memory.",
      "rejected":"Study all night in one sitting and skip sleep to save time."},
    {"prompt":"What is AI in simple words?",
      "chosen":"AI is when computers do tasks that normally need human intelligence, like understanding language or recognizing images.",
      "rejected":"AI is magic inside a computer that knows everything without code."},
    {"prompt":"How to stay safe online?",
      "chosen":"Use strong unique passwords, enable 2FA, avoid unknown links, and keep your software updated.",
      "rejected":"Reuse the same password everywhere and click unknown links to check them."},
]
dpo_ds = Dataset.from_list(raw); dpo_ds


Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 4
})

## 4) Load tokenizer & policy model (FP32)

In [4]:

base_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

# Free any previous model
try:
    del policy_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except NameError:
    pass

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

policy_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float32,   # FP32 for stability
)
policy_model.config.use_cache = False


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## 5) Apply LoRA adapters

In [11]:

lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"]
)


## 6) TrainingArguments (FP32, reference‑free)

In [5]:

from dataclasses import fields

BATCH = 16
base_kwargs = dict(
    output_dir="smollm2-135m-dpo",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=BATCH,
    learning_rate=1e-4,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=200,
    save_total_limit=1,
    bf16=False,
    fp16=False,                  # ensure no AMP
    report_to="none",
)

has_eval = "evaluation_strategy" in {f.name for f in fields(TrainingArguments)}
args = (TrainingArguments(evaluation_strategy="steps", eval_steps=50, **base_kwargs)
        if has_eval else TrainingArguments(**base_kwargs))


## 7) Initialize DPOTrainer (reference‑free + LoRA)

In [7]:

_policy = policy_model.module if hasattr(policy_model, "module") else policy_model

dpo_trainer = DPOTrainer(
    model=_policy,
    ref_model=None,                # reference-free with LoRA
    beta=0.1,
    args=args,                     # TrainingArguments for TRL 0.7.x
    train_dataset=dpo_ds,
    eval_dataset=None,
    tokenizer=tokenizer,
    peft_config=lora_cfg,
    max_length=256,
    max_prompt_length=128,
)


/usr/local/lib/python3.12/dist-packages/trl/trainer/dpo_trainer.py:158: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


## 8) Train

In [8]:

dpo_trainer.train()


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


TrainOutput(global_step=2, training_loss=0.17013868689537048, metrics={'train_runtime': 2.5812, 'train_samples_per_second': 3.099, 'train_steps_per_second': 0.775, 'total_flos': 0.0, 'train_loss': 0.17013868689537048, 'epoch': 2.0})

## 9) Test the tuned model

In [9]:

def generate(prompt, max_new_tokens=120):
    model = dpo_trainer.model
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.8, top_p=0.9)
    print(tokenizer.decode(out[0], skip_special_tokens=True))

generate("### Instruction:\nGive two tips to study better.\n\n### Response:\n")


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


### Instruction:
Give two tips to study better.

### Response:

"First, study by breaking it down into smaller parts. For example, if you're studying for a test, break it down into different subjects or topics to study. This will help you to understand the concepts better and make it easier to understand what to expect.

Second, practice regularly. This will help you to memorize key information, and to feel more confident when you do it."


## 10) Save the LoRA adapter

In [10]:

adapter_dir = "smollm2-135m-dpo-lora-adapter"
dpo_trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print("Saved DPO LoRA adapter to:", adapter_dir)


Saved DPO LoRA adapter to: smollm2-135m-dpo-lora-adapter
